# Portfolio Optimization Project

### CS/ECE/ISyE 524 — Introduction to Optimization — Fall 2024

### Project Members: Pawin Linmaneechote, Vincent Philavong, Jiajie Yao

## Table of Contents
1. Introduction  
    1.1. Project Motivation  
    1.2. Data Processing  
2. Mathematical Model  
    2.1 Minimum Volitaty Optimization (MVO) Model with L2 Regularlization (QP)  
    2.2 Markowitz Portfolio with Diversification (QP)  
    2.3 Sharpe Ratio with L2 Regularlization (NLP)  
3. Solution  
    3.1 Trade Off Analysis  
    3.2 Sensitivity Analysis  
4. Results and Discussion
5. Conclusion
6. Appendix
7. References

# 1. Introduction
## 1.1. Project Motivation
Investing is about making smart decisions to balance the trade-offs between risk and reward. In this project, we explore portfolio optimization by analyzing financial data from Yahoo Finance. We’ve chosen five asset sectors - Technology, Financials, Healthcare, Communication Services, and Industrials - to build a diversified portfolio. This selection spreads risk across industries while aiming to capture each sector’s growth potential. The list of the stock sectors and the individual stocks can be seen below.
- Technology
    - Apple (AAPL)
    - Uber (UBER)
    - Microsoft (MSFT)
    - NVIDIA (NVDA)
    - Advanced Micro Devices (AMD)
- Financial
    - Discover (DFS)
    - JP Morgan (๋JPM)
    - Synchrony (SYF)
    - American Express (AXP)
    - Goldman Sachs (GS)
- Health Care
    - Pfizer (PFE)
    - Johnson & Johnson (JNJ)
    - Merck Sharp & Dohme (MRK)
    - United Health Group (UNH)
    - ABBVie (ABBV)
- Communication Services
    - Meta (META)
    - Google (GOOG)
    - Netflix (NFLX)
    - T-Mobile (TMUS)
    - The Walt Disney Services (DIS)
- Industrials
    - Caterpillar (CAT)
    - RTX Coorporation (RTX)
    - The Boeing Company (BA)
    - Union Pacific Corperation (UNP)
    - Eaton Corporation PLC (ETN)

Our project focuses on optimizing three models. The first model, also known as the Minimum Volitaty Optimization (MVO), addressing the challenge of balancing variance in addition to diversification by analyzing the performance and relationships between the selected stocks. The second, known as the, Markowitz Model, builds on the baseline by of the variance model by tranforming the model into a trade-off between returns, variance, and diversification, which we believe will give a better result for the allocations. For the third model, we introduce a Sharpe Ratio model, focusing on maximizing the Sharpe Ratio with diversification to evaluate the effeciency of the stock allocations.

Portfolio optimization has been a topic of interest for investors and researchers for decades. It’s rooted in the uncertainty of financial markets, where both short-term gains and long-term stability needed careful evaluation. The economist, Harry Markowitz, revolutionized this field in 1952 with his paper Portfolio Selection, introducing diversification as a way to reduce risk (Team Investopedia, 2023). His work laid the groundwork for modern portfolio theory (MPT) and earned him a Nobel Prize. Yet, as markets continue to evolve, ongoing research is critical to adapting strategies for changing conditions like volatility, economic shifts, and new investor behaviors.

This report is structured into four sections: the Mathematical Model, where we define the decision variables, constraints, and objective function; the Solution, where we implement the model in Julia; Results and Discussion, where we present and interpret the outcomes; and the Conclusion, summarizing our findings and proposing future directions.

## 1.2 Data Preparation

In [ ]:
using CSV, DataFrames, Dates, Statistics, JuMP, Ipopt, PyPlot, LinearAlgebra

# ---------------------------
# SECTION 1: Read CSV Files and Extract Equity Percent 
# ---------------------------

directory_path = "data_files/"
stock_sector = CSV.read("Stock Sheet.csv", DataFrame)
rename!(stock_sector, Symbol("Ticker") => :StockName) 

result_df = DataFrame(StockName = String[], OpenStock = Float64[], CloseStock = Float64[], EquityPercent = Float64[])

foreach(file -> begin
    if endswith(file, ".csv")
        stock_df = CSV.read(joinpath(directory_path, file), DataFrame)
        sort!(stock_df, :Date)
        
        stock_name = splitext(basename(file))[1]
        open_price, close_price = stock_df.Open[1], stock_df.Close[end]
        equity_percent = ((close_price - open_price) / open_price) * 100

        push!(result_df, (stock_name, open_price, close_price, equity_percent))
    end
end, readdir(directory_path))

final_df = leftjoin(result_df, stock_sector[:, [:StockName, :StockSector]], on = :StockName)
select!(final_df, [:StockName, :OpenStock, :CloseStock, :EquityPercent, :StockSector])

println(final_df)

# ---------------------------
# SECTION 2: Extract Daily Returns for Each Stock 
# ---------------------------

returns_df = DataFrame(Date = Date[])

for file in readdir(directory_path)
    if endswith(file, ".csv")
        file_path = joinpath(directory_path, file)
        stock_df = CSV.read(file_path, DataFrame)
        stock_df.Date = Date.(string.(stock_df.Date), "yyyy-mm-dd")
        sort!(stock_df, :Date)
        stock_name = splitext(basename(file))[1]
        stock_df[!, :DailyReturn] = [missing; diff(log.(stock_df.Close))]
        stock_returns = stock_df[:, [:Date, :DailyReturn]]
        rename!(stock_returns, :DailyReturn => stock_name)
        if nrow(returns_df) == 0
            returns_df = stock_returns
        else
            returns_df = outerjoin(returns_df, stock_returns, on = :Date)
        end
    end
end

complete_cases = dropmissing(returns_df, disallowmissing=true)
CSV.write("complete_cases.csv", complete_cases)

# ---------------------------
# SECTION 3: Calculate Asset Metrics and Sector Mappings 
# ---------------------------

asset_names = setdiff(Symbol.(names(complete_cases)), [:Date])
returns_matrix = Matrix(complete_cases[:, asset_names])

trading_days = 252
μ = (1 .+ mean(returns_matrix, dims=1)) .^ trading_days .- 1 
μ = vec(μ)  
Σ = cov(returns_matrix) * trading_days 

println()
println("Length of μ: ", length(μ))
println("Size of Σ: ", size(Σ))
println("Number of assets (n_assets): ", length(μ))

stock_sheet = CSV.read("Stock Sheet.csv", DataFrame)
ticker_to_sector = Dict(stock_sheet.Ticker .=> stock_sheet.StockSector)

tickers = ["AAPL", "ABBV", "AMD", "AXP", "BA", "CAT", "DFS", "DIS", 
           "ETN", "GOOG", "GS", "JNJ", "JPM", "META", "MRK", "MSFT", 
           "NFLX", "NVDA", "PFE", "RTX", "SYF", "TMUS", "UBER", 
           "UNH", "UNP"]

missing_tickers = filter(ticker -> !haskey(ticker_to_sector, ticker), tickers)
!isempty(missing_tickers) && println("The following tickers are missing in the stock sheet: ", missing_tickers)

sector_id = Dict{String, Vector{Int}}()
for (i, ticker) in enumerate(tickers)
    if haskey(ticker_to_sector, ticker)
        sector = ticker_to_sector[ticker]
        push!(get!(sector_id, sector, Int[]), i)
    end
end

println("Sector-to-index mapping: ", sector_id)

# 2. Mathematical Model

## 2.1 Assumptions

Our assumptions for this portfolio optimization is based on several key principles

1. Diversification: We assume that selecting stocks from diverse sections will reduce the overall portfolio risk while maximize the return opportunities from each sector
2. Model Framework:
    - The baseline model is built on the Minimum Volitaty Optimization Model, which assumes that the historical returns and covariances would be sufficient to estimate risk and return
    - The Markowitz model assumes that penalizing extreme portfolio weights would increase the diversification of our portfolio
    - The Sharpe model assumes that the Sharpe ratio would balance the risk and return for an investor-focused outcome
3. Market Dynamics: We assume that market volatility is stab;e or fluctuate within a certain range during our analysis
4. Investor Behavior: The models we use will assume a rational investor with the objective to maximize returns while minimizing risk.

## 2.2 Models 

We established allocation percentage constraints for our models to ensure balanced investments. For individual stocks, we set a lower bound of 0.02 and an upper bound of 0.1 for their allocations. Additionally, for each sector, we implemented a lower bound of 0.1 and an upper bound of 0.4 on the total allocation.

### 2.2.1 Minimum Volitaty Optimization Model with Diversification Term (QP)

In our initial model, we aim to optimize the portfolio by balancing the trade-off between maximizing returns and minimizing risk. This is achieved using a diversification penalty factor ($\gamma$), which controls the level of emphasis placed on risk relative to returns. The optimization problem is mathematically defined as follows:

#### Parameters:
- $\Sigma$: Covariance matrix of asset returns.
- $\mu$: Vector of expected returns for each asset.
- $\gamma$: Diversification penalty factor.
- $n_{assets}$: Number of assets in the portfolio
- $ \text{sector} $: Mapping of sectors to asset indices.


#### Decision Variables:

- $x_{\text{i}}:$ Percentage of the total budget allocated to asset $ i, \quad i \in \{1, 2,\ldots, n_{\text{assets}}\}. $

#### Constraints:  
1. Bounds on Allocation  
\begin{aligned}
0.02 \leq x_i \leq 0.10, \quad \forall i \in \{1, 2, \ldots, n_{\text{assets}}\}.
\end{aligned}
2. Budget Constraint
\begin{aligned}  
\sum_{i=1}^{n_{\text{assets}}} x_i = 1.
\end{aligned}
3. Sector Allocation Constraints
\begin{aligned}
0.10 \leq \sum_{i \in \text{sector } j} x_i \leq 0.40, \quad \forall j \in \{1, 2, \ldots, n_{\text{sectors}}\}.
\end{aligned}

#### Objective Functions:
\begin{aligned}
\text{Minimize} \quad x^\top \Sigma x + \gamma \sum_{i=1}^{n_{\text{assets}}} x_i^2
\end{aligned}  
Where:
- $ x^\top \Sigma x $ : Portfolio variance (risk).  
- $ \sum_{i=1}^{n_{\text{assets}}} x_i^2 $ : Penalty term to promote diversification.


#### Final Model:
\begin{aligned}
\text{Minimize} \quad & x^\top \Sigma x + \gamma \sum_{i=1}^{n_{\text{assets}}} x_i^2 \\
\text{Subject to} \quad & \sum_{i=1}^{n_{\text{assets}}} x_i = 1, \\
& 0.02 \leq x_i \leq 0.1 \quad \forall i \in \{1, 2, \ldots, n_{\text{assets}}\}, \\
& 0.10 \leq \sum_{i \in \text{sector} j} x_i \leq 0.40 \quad \forall j \in \{1, 2, \ldots, n_{\text{sectors}}\}.
\end{aligned}


### 2.2.2 Markowitz Portfolio with Diversification (QP)

While experimenting with initial model, we found out that NVDA overpowers all the other stocks in returns. So we decided to add a trade-off for risk and diversification by adding a Ridge regularization term. This is beneficial as Ridge regularization is a smoothing algorithm which results in values being closer in magnitude. While the initial model focused solely on maximizing returns and minimizing risk, this updated formulation includes a diversification penalty factor ($\gamma$) that penalizes the concentration of portfolio weights. We also have a risk aversion parameter ($\lambda $) to control the trade-off between the return and the risk. The optimization problem is defined as follows:

#### Parameters:
- $\Sigma$: Covariance matrix of asset returns.
- $\mu$: Vector of expected returns for each asset.
- $\gamma$: Diversification penalty factor.
- $n_{assets}$: Number of assets in the portfolio
- $ \text{sector} $: Mapping of sectors to asset indices.
- $\lambda $: Risk aversion parameter that controls the trade-off between return and risk.


#### Decision Variables:

- $x_{\text{i}}:$ Percentage of the total budget allocated to asset $ i, \quad i \in \{1, 2,\ldots, n_{\text{assets}}\} $

#### Constraints:  
1. Bounds on Allocation  
\begin{aligned}
0.02 \leq x_i \leq 0.10, \quad \forall i \in \{1, 2, \ldots, n_{\text{assets}}\}
\end{aligned}
2. Budget Constraint
\begin{aligned}  
\sum_{i=1}^{n_{\text{assets}}} x_i = 1
\end{aligned}
3. Sector Allocation Constraints
\begin{aligned}
0.10 \leq \sum_{i \in \text{sector } j} x_i \leq 0.40, \quad \forall j \in \{1, 2, \ldots, n_{\text{sectors}}\}
\end{aligned}

#### Objective Function
\begin{aligned}
\text{Maximize} \quad \mu^\top x - \lambda (x^\top \Sigma x) - \gamma \sum_{i=1}^{n_{\text{assets}}} x_i^2
\end{aligned}
Where:
- $ \mu^\top x $ : Expected return of the portfolio.
- $ x^\top \Sigma x $ : Portfolio variance (risk).
- $ \sum_{i=1}^{n_{\text{assets}}} x_i^2 $ : Penalty term to promote diversification.

#### Final Model
\begin{aligned}
\text{Maximize} \quad & \mu^\top x - \lambda (x^\top \Sigma x) - \gamma \sum_{i=1}^{n_{\text{assets}}} x_i^2 \\
\text{Subject to} \quad & \sum_{i=1}^{n_{\text{assets}}} x_i = 1, \\
& 0.02 \leq x_i \leq 0.1 \quad \forall i \in \{1, 2, \ldots, n_{\text{assets}}\}, \\
& 0.10 \leq \sum_{i \in \text{sector} j} x_i \leq 0.40 \quad \forall j \in \{1, 2, \ldots, n_{\text{sectors}}\}.
\end{aligned}


### 2.2.3 Sharpe Ratio with Diversification (QP)

In this model, we aim to optimize the Sharpe ratio, a measure of risk-adjusted return where we believe that it would increase the overall efficiency. The Sharpe model includes a risk-free rate ($r_f$) and a divserification penalty factor ($\gamma$). The mathematical formulation is as follows:

#### Parameters:
- $\Sigma$: Covariance matrix of asset returns.
- $\mu$: Vector of expected returns for each asset.
- $\gamma$: Diversification penalty factor.
- $n_{assets}$: Number of assets in the portfolio
- $r_f$ Risk-free rate (set to 0 in this model)
- $ \text{sector} $: Mapping of sectors to asset indices.

#### Decision Variables:

- $x_{\text{i}}:$ Percentage of the total budget allocated to asset $ i, \quad i \in \{1, 2,\ldots, n_{\text{assets}}\} $

#### Constraints:  
1. Bounds on Allocation  
\begin{aligned}
0.02 \leq x_i \leq 0.10, \quad \forall i \in \{1, 2, \ldots, n_{\text{assets}}\}
\end{aligned}
2. Budget Constraint
\begin{aligned}  
\sum_{i=1}^{n_{\text{assets}}} x_i = 1
\end{aligned}
3. Sector Allocation Constraints
\begin{aligned}
0.10 \leq \sum_{i \in \text{sector } j} x_i \leq 0.40, \quad \forall j \in \{1, 2, \ldots, n_{\text{sectors}}\}
\end{aligned}

#### Objective Function
\begin{aligned}
\text{Maximize} \quad \frac{\mu^\top x - r_f}{\sqrt{x^\top \Sigma x}} - \gamma \sum_{i=1}^{n_{\text{assets}}} x_i^2
\end{aligned}
Where:
- $ \mu^\top x $ : Expected portfolio return.
- $ r_f $ : Risk-free rate.
- $ \sqrt{x^\top \Sigma x} $ : Portfolio standard deviation (risk).
- $ \gamma \sum_{i=1}^{n_{\text{assets}}} x_i^2 $ : Penalty term to encourage diversification.

#### Final Model
\begin{aligned}
\text{Maximize} \quad & \mu^\top x - \lambda (x^\top \Sigma x) - \gamma \sum_{i=1}^{n_{\text{assets}}} x_i^2 \\
\text{Subject to} \quad & \sum_{i=1}^{n_{\text{assets}}} x_i = 1, \\
& 0.02 \leq x_i \leq 0.1 \quad \forall i \in \{1, 2, \ldots, n_{\text{assets}}\}, \\
& 0.10 \leq \sum_{i \in \text{sector} j} x_i \leq 0.40 \quad \forall j \in \{1, 2, \ldots, n_{\text{sectors}}\}.
\end{aligned}


# 3. Solutions

## 3.1 Minimum Volitaty Optimization Model with L2 Regularlization (QP)

In [ ]:
# Assumes Data Processing has ran
function MVO(γ)
    r_f = 0
    # Create optimization model
    m = Model(Ipopt.Optimizer)
    set_optimizer_attribute(m, "print_level", 0)

    # Decision variables
    @variable(m, 0.02 <= x[1:n_assets] <= 0.1) # Decision variable for asset allocation | asset allocation bounded in range(2%, 10%)

    # Constraints
    @constraint(m, sum(x) == 1) # Budget Constraints, ensures asset allocations are equal to 100%.
    for (sector, indices) in sector_id
        @constraint(m, 0.1 <= sum(x[i] for i in indices) <= 0.4) # Sector constraint bouned in range(10%, 40%).
    end

    # Objective
    @objective(m, Min, x' * Σ * x + γ * sum(x[i]^2 for i in 1:n_assets)) # Minimize Volitaty with a trade-off for diversification

    # Solve the model
    optimize!(m)
    status = termination_status(m)

    if status == MOI.OPTIMAL || status == MOI.LOCALLY_SOLVED
        # Extract optimal portfolio weights
        portfolio_weights = value.(x)

        # Compute portfolio return and risk
        portfolio_return = μ' * portfolio_weights  # Annual expected return
        portfolio_risk = sqrt(portfolio_weights' * Σ * portfolio_weights)  # Annual standard deviation
        sharpe_ratio = (portfolio_return - r_f) / portfolio_risk
        return(portfolio_weights, portfolio_return, portfolio_risk, sharpe_ratio)
    else
        println("Model infeasible")
    end
end

## 3.2 Markowitz Portfolio with Diversification (QP)

In [ ]:
function Markowitz(γ)
    # Create optimization model
    λ = 1
    r_f = 0
    m = Model(Ipopt.Optimizer)
    set_optimizer_attribute(m, "print_level", 0)

    # Define variables
    @variable(m, 0.02 <= x[1:n_assets] <= 0.1)
    @constraint(m, sum(x) == 1)

    for (sector, indices) in sector_id
        @constraint(m, 0.1 <= sum(x[i] for i in indices) <= 0.4)
    end


    @objective(m, Max, μ' * x - λ * (x' * Σ * x) - γ * sum(x[i]^2 for i in 1:n_assets))

    # Solve the model
    optimize!(m)

    # Check solution status
    status = termination_status(m)

    if status == MOI.OPTIMAL || status == MOI.LOCALLY_SOLVED
        portfolio_weights = value.(x)
        portfolio_return = μ' * portfolio_weights  # Annual expected return
        portfolio_risk = sqrt(portfolio_weights' * Σ * portfolio_weights)  # Annual standard deviation
        sharpe_ratio = (portfolio_return - r_f) / portfolio_risk
        return(portfolio_weights, portfolio_return, portfolio_risk, sharpe_ratio)
    else
        println("Model is infeasible.")
    end
end

## 3.3 Sharpe Ratio with L2 Regularlization (NLP)

In [ ]:
# Create optimization model
function Sharpe(γ)
    r_f = 0    # Risk-free rate
    
    m = Model(Ipopt.Optimizer)
    set_optimizer_attribute(m, "print_level", 0)

    # Define variables
    @variable(m, 0.02 <= x[1:n_assets] <= 0.1)
    @constraint(m, sum(x) == 1)

    for (sector, indices) in sector_id
        @constraint(m, 0.1 <= sum(x[i] for i in indices) <= 0.4)
    end

    # Objective function: maximize Sharpe ratio - γ * sum of squares of weights
    @objective(m, Max, (dot(μ, x) - r_f) / sqrt(x' * Σ * x) - γ * sum(x[i]^2 for i in 1:n_assets))

    # Solve the model
    optimize!(m)

    # Check the termination status
    status = termination_status(m)

    if status == MOI.OPTIMAL || status == MOI.LOCALLY_SOLVED
        portfolio_weights = value.(x)
        portfolio_return = dot(μ, portfolio_weights)  # Annual expected return
        portfolio_risk = sqrt(dot(portfolio_weights, Σ * portfolio_weights))  # Annual standard deviation
        sharpe_ratio = (portfolio_return - r_f) / portfolio_risk
        return(portfolio_weights, portfolio_return, portfolio_risk, sharpe_ratio)
    else
        println("Model is infeasible.")
    end
end

# 4. Results and Discussion

## 4.1 Model Allocation Percentages Proportions for each Sector and Asset

TODO: analysis motivation

In [ ]:
y = 1 # Gamma input
# Call the functions once and store the results
mvo_result = MVO(y)
markowitz_result = Markowitz(y)
sharpe_result = Sharpe(y)

# Use the stored results to build the dictionary
portfolio_results = Dict(
    :MVO => (weights = mvo_result[1], ret = mvo_result[2], risk = mvo_result[3], sharperatio = mvo_result[4]),
    :Markowitz => (weights = markowitz_result[1], ret = markowitz_result[2], risk = markowitz_result[3], sharperatio1 = markowitz_result[4]),
    :Sharpe => (weights = sharpe_result[1], ret = sharpe_result[2], risk = sharpe_result[3], sharperatio = sharpe_result[4])
)


# --- Plotting Sector Weights for Each Model ---
model_order = [:MVO, :Markowitz, :Sharpe]

# Set up subplots: 1 row, number of models as columns
num_models = length(model_order)
fig, axes = subplots(1, num_models, figsize=(15, 6))  # Adjust figsize as needed

# Iterate through the models in the desired order
for (i, model_key) in enumerate(model_order)
    result = portfolio_results[model_key]
    
    # Calculate sector weights for the current model
    sector_weights = Dict()
    for (sector, indices) in sector_id
        total_weight = sum(result.weights[i] for i in indices)
        sector_weights[sector] = total_weight
    end

    # Plot pie chart on the corresponding subplot
    ax = axes[i]  # Select the subplot
    ax.pie(collect(values(sector_weights)), 
           labels=collect(keys(sector_weights)),
           autopct="%1.1f%%")
    title_text = "Portfolio Sector Distribution: $model_key (γ = $y)"
    ax.set_title(title_text)
    
end

# Adjust layout to prevent overlap and show the plots
tight_layout()
show()

TODO: ANALYSIS EXPLAINATION

In [ ]:
figure(figsize=(10, 15))  # Adjust the figure size for a vertical layout

# Colors for different portfolios
portfolio_names = collect(keys(portfolio_results))
colors = ["blue", "green", "red"]  # Ensure the number of colors matches the number of portfolios

# Plotting the weights for each portfolio
for (i, portfolio_name) in enumerate(portfolio_names)
    subplot(length(portfolio_results), 1, i)  # Change to vertical layout
    portfolio_weights = portfolio_results[portfolio_name][:weights]
    bar(1:n_assets, portfolio_weights, color=colors[i], alpha=0.7)
    xticks(1:n_assets, asset_names, rotation=90)
    xlabel("Assets")
    ylabel("Portfolio Weight")
    title("$portfolio_name Portfolio Weights")
    grid(true)
    tight_layout()
end

show()


## 4.2 Cumulative Weight Distribution Analysis

ANALYSIS MOTIVATION

In [ ]:
figure(figsize=(18, 18))  # Adjust figure size to accommodate three subplots

# Plot for MVO Portfolio Cumulative Weight Distribution
subplot(3, 1, 1)
sorted_weights = sort(collect(zip(final_df.StockName, mvo_result[1])), by=x->x[2], rev=true)
weights = [w[2] for w in sorted_weights]
cumsum_weights = cumsum(weights)
plot(1:length(weights), cumsum_weights, "bo-")  # Blue color for MVO
xticks(1:length(weights), [w[1] for w in sorted_weights], rotation=45, ha="right")
xlabel("Stocks")
ylabel("Cumulative Weight")
title("MVO Portfolio Cumulative Weight Distribution")
grid(true)
tight_layout()

# Plot for Markowitz Portfolio Cumulative Weight Distribution
subplot(3, 1, 2)
sorted_weights = sort(collect(zip(final_df.StockName, markowitz_result[1])), by=x->x[2], rev=true)
weights = [w[2] for w in sorted_weights]
cumsum_weights = cumsum(weights)
plot(1:length(weights), cumsum_weights, "go-")  # Green color for Markowitz
xticks(1:length(weights), [w[1] for w in sorted_weights], rotation=45, ha="right")
xlabel("Stocks")
ylabel("Cumulative Weight")
title("Markowitz Portfolio Cumulative Weight Distribution")
grid(true)
tight_layout()

# Plot for Sharpe Portfolio Cumulative Weight Distribution
subplot(3, 1, 3)
sorted_weights = sort(collect(zip(final_df.StockName, sharpe_result[1])), by=x->x[2], rev=true)
weights = [w[2] for w in sorted_weights]
cumsum_weights = cumsum(weights)
plot(1:length(weights), cumsum_weights, "ro-")  # Red color for Sharpe
xticks(1:length(weights), [w[1] for w in sorted_weights], rotation=45, ha="right")
xlabel("Stocks")
ylabel("Cumulative Weight")
title("Sharpe Portfolio Cumulative Weight Distribution")
grid(true)
tight_layout()

savefig("cumulative_weights_combined.png")

ANALYSIS EXPLANTATION

## 4.3 Diversification Sensitivity Analysis

ANALYSIS MOTIVATION

In [ ]:
γ_values = 0:1:100
models = ["MVO", "Markowitz", "Sharpe"]
results = []

# Define colors for models
model_colors = Dict("MVO" => "blue", "Markowitz" => "green", "Sharpe" => "red")

# Collect results for each model and γ
for model in models
    for γ in γ_values
        port = eval(Symbol(model))(γ)  # MVO and Sharpe use only γ
        if !isnothing(port)
            portfolio_weights, portfolio_return, portfolio_risk, sharpe_ratio = port
            portfolio_variance = portfolio_risk^2  # Variance is the square of risk
            push!(results, (model, γ, portfolio_return, portfolio_variance))
        end
    end
end

# Create subplots for Portfolio Return and Portfolio Variance
figure(figsize=(14, 5))

# Plot 1: Portfolio Return
subplot(1, 2, 1)
for model in models
    # Filter results for the current model
    model_results = filter(r -> r[1] == model, results)
    γ_vals = [r[2] for r in model_results]
    ret_vals = [r[3] for r in model_results]

    # Fit a polynomial to the data
    degree = 3
    X = hcat([γ_vals.^k for k in degree:-1:0]...)
    coeffs = X \ ret_vals

    # Polynomial function
    poly_fit = x -> sum(coeffs[i] * x^(degree - i + 1) for i in 1:length(coeffs))

    # Scatter plot and fitted polynomial
    scatter(γ_vals, ret_vals, alpha=0, color=model_colors[model])
    x_fit = range(minimum(γ_vals), stop=maximum(γ_vals), length=200)
    y_fit = [poly_fit(x) for x in x_fit]
    plot(x_fit, y_fit, label="Polynomial Fit $model", lw=2, color=model_colors[model])
end
xlabel("Diversification Penalty (γ)")
ylabel("Portfolio Return")
title("Diversification Penalty (γ) vs Portfolio Return")
legend()
grid(true)

# Plot 2: Portfolio Variance
subplot(1, 2, 2)
for model in models
    # Filter results for the current model
    model_results = filter(r -> r[1] == model, results)
    γ_vals = [r[2] for r in model_results]
    variance_vals = [r[4] for r in model_results]

    # Fit a polynomial to the data
    degree = 3
    X = hcat([γ_vals.^k for k in degree:-1:0]...)
    coeffs = X \ variance_vals

    # Polynomial function
    poly_fit = x -> sum(coeffs[i] * x^(degree - i + 1) for i in 1:length(coeffs))

    # Scatter plot and fitted polynomial
    scatter(γ_vals, variance_vals, alpha=0, color=model_colors[model])
    x_fit = range(minimum(γ_vals), stop=maximum(γ_vals), length=200)
    y_fit = [poly_fit(x) for x in x_fit]
    plot(x_fit, y_fit, label="Polynomial Fit $model", lw=2, color=model_colors[model])
end
xlabel("Diversification Penalty (γ)")
ylabel("Portfolio Variance")
title("Diversification Penalty (γ) vs Portfolio Variance")
legend()
grid(true)

# Adjust layout and show the plots
tight_layout()
show()


ANALYSIS EXPLANATION

## 4.4 Last Analysis

##  Sensitivity Analysis

[make changes here]

The sensitivity analysis reveals complex interactions between the minimum return requirement (R_min) and risk aversion coefficient (λ). When R_min is low (0.001), the portfolio exhibits high sensitivity to changes in λ, with returns decreasing from 337.68% to 24.15% and risk reducing from 44.15% to 0.06% as λ increases from 1 to 100. This demonstrates the dominant effect of risk aversion when return constraints are not binding.

However, as R_min increases, we observe a distinctive "threshold effect" where the minimum return constraint becomes binding, limiting the impact of risk aversion. For R_min = 2.0, the portfolio maintains a constant return of 200% and risk of 14.7% when λ ≥ 10, while for R_min = 3.0, the portfolio stabilizes at a 300% return and 34.52% risk when λ ≥ 5. This pattern indicates that higher minimum return requirements effectively constrain the risk-minimization objective, creating a floor for both return and risk levels regardless of further increases in risk aversion.

These findings suggest that the effectiveness of risk control through λ adjustment is significantly influenced by the minimum return requirement, with higher R_min values reducing the available risk-return trade-off space. This interaction provides crucial insights for portfolio managers in setting appropriate risk-return targets and understanding their feasible investment options.


# 5. Conclusion

# 6. References

- Team, T. I. (2023, August 29). Modern portfolio theory: What MPT is and how investors use it. Investopedia. https://www.investopedia.com/terms/m/modernportfoliotheory.asp?utm_source=chatgpt.com 